### Тематическое моделирование 



__Важно__: Во второй половине ноутбука мы будем работать с библиотекой BigARTM, запускать которую удобнее всего в google colab (ниже будет код для установки библиотеки в colab), или с помощью Docker-образа. Если вы планируете работать с использованием Docker, сперва запустите его и работайте с ноутбуком внутри окружения, чтобы не делать предобработку данных дважды. Инструкция ниже.


Есть несколько сценариев работы с BigARTM, среди которых:

- установка библиотеки для Питона: инструкция [здесь](https://bigartm.readthedocs.io/en/stable/installation/index.html)

- запуск Docker образа с библиотекой

Если вы предпочтете второй сценарий, вам нужно:
1. Скачать и установить [Docker](https://www.docker.com/get-started) для своей ОС. Запустить Docker. 
2. Скачать образ (Dockerfile), в котором устанавливается большинство библиотек для анализа данных, Jupyter Notebook и сама BigARTM. Ссылка на файл лежит рядом со ссылкой на этот ноутбук в материалах недели.
3. Собрать образ: это можно сделать через командную строку (находясь в папке, где лежит скачанный в п.2 Dockerfile):

$ docker build -t artm .

$ docker run -p 8888:8888 artm

После выполнения этих двух команд запустится Jupyter Notebook, в командной строке будет написан путь к нему в таком формате:

Or copy and paste one of these URLs:
        http://de0ad378d760:8888/?token=5897bc15b494ca17f0d996763a8d05a71f1170af3455e96d
     or http://127.0.0.1:8888/?token=5897bc15b494ca17f0d996763a8d05a71f1170af3455e96d   

Откройте любой из URL, там поднят Jupyter Notebook с уже установленными библиотеками.

При работе не забывайте, что все ноутбуки, которые вы создаете внутри этого Docker образа, исчезнут, как только вы остановите Docker, поэтому перед окончанием работы скачайте ноутбук с результатом себе на компьютер.


Построим тематические модели для корпуса новостей Lenta.ru, с которым мы уже работали ранее.

In [45]:
# !wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [46]:
# !pip install corus

In [47]:
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [48]:
data = [(record.title, record.topic, record.text, record.tags) for record in records]

In [49]:
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

<ipython-input-49-5b7176a9d163>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [50]:
data = pd.DataFrame(data, columns=['title','topic','text','tags'])

In [51]:
data.sample(3)

,title,topic,text,tags
683684,РТР запускает новый информационный канал,Интернет и СМИ,"26 августа в эфире телеканала РТР появится новая программа - ""Доброе утро Россия"", сообщает РИА ""Новости"". Программа будет выходить по утрам с шести до девяти часов. Она будет рассчитана на людей ведущих активный образ жизни. В частности, каждые 15 минут в эфир будут выходить поочередно - московские и российские выпуски новостей. Отдельно в программе предусмотрены ""Всемирные новости"". Кроме того, в программе будут рубрики ""Экспертиза РТР"", ""Дорожный патруль"" и ""Афиша"". Вести программу будут поочередно три ведущих - Эрнест Мацкявичус и Олеся Лосева, Алексей Лысенков и Ольга Иванцова, а также Илья Легостаев с журналисткой, имя которой пока не известно.",Все
64641,Женская сборная России по футболу вылетела с чемпионата Европы,Спорт,"Женская сборная России по футболу проиграла команде Германии в матче третьего тура группового этапа чемпионата Европы и покинула турнир. Об этом сообщает корреспондент «Ленты.ру». Встреча состоялась вечером во вторник, 25 июля, в голландском Утрехте и завершилась со счетом 2:0 в пользу немок. Забитыми мячами отличились Бабетт Петер (десятая минута) и Дженифер Марожан (56-я). В параллельном матче сборная Италии оказалась сильнее девушек из Швеции — 3:2. Сборная России заняла третье место в группе В с тремя набранными очками. Первое и второе места, дающее право на выход в плей-офф, осталось за немками и шведками соответственно. Италия замкнула турнирную таблицу. Чемпионат Европы проходит с 16 июля по 6 августа в Нидерландах. Действующим победителем турнира является сборная Германии.",Футбол
202113,На «Марш правды» в Киеве вышли несколько сотен человек,Бывший СССР,"На «Марш правды» в Киеве, организованный радикальным «Правым сектором», вышли несколько сотен человек. Об этом сообщил телеканал «112 Украина». Еще до начала шествия произошла драка между участниками митинга и представителями правоохранительных органов. Был задержан активист с символикой «Украинского братства», у него нашли бутылку с бензином. После инцидента бойцы Нацгвардии, задействованные в охране мероприятия, начали обыскивать участников шествия. В самом «Правом секторе» также объявили о запрете на использование любой пиротехники, отметив, что это политическая акция, а не штурм. Среди участников марша замечены люди в форме Добровольческого украинского корпуса — боевого подразделения «Правого сектора». В шествии принимали участие и футбольные фанаты, которые, как отмечается, ушли после начала обысков на наличие пиротехники и горючих веществ. Участники собрались возле метро «Арсенальная», откуда по улице Московской двинулись к зданию Генпрокуратуры Украины. Возле ГПУ к ним вышел представитель ведомства, которому митингующие объявили, что «третьего Майдана не будет, будут сразу расстрелы», а затем выдвинулись в сторону МВД. Возле министерства манифестанты объявили, что «пришли не за Европой, а своими правами», и выразили недовольство работой милиции. Как сообщает УНН, некоторые из участников марша, которые остались возле ГПУ, изготовили коктейль Молотова. Сами митингующие назвали его символическим напоминанием властям о событиях Майдана. Изначально «Марш правды» был заявлен «Правым сектором» как протест против лиц, которые заняли руководящие должности в результате Евромайдана. Утверждалось, что они (новые власти) «сдали интересы Украины» и сделали население страны бедным. Хронику мероприятия можно посмотреть, например, здесь. «Правый сектор» — национал-радикальная организация, была одной из движущих боевых сил Майдана во время протестов в ноябре 2013 — феврале 2014 года. В России деятельность организации официально была запрещена, ее признали экстремистской.",Украина


In [52]:
len(data)

739350

In [53]:
data.topic.value_counts().head(15)

Россия               160518
Мир                  136680
Экономика            79538 
Спорт                64421 
Культура             53803 
Бывший СССР          53402 
Наука и техника      53136 
Интернет и СМИ       44675 
Из жизни             27611 
Дом                  21734 
Силовые структуры    19596 
Ценности             7766  
Бизнес               7399  
Путешествия          6408  
69-я параллель       1268  
Name: topic, dtype: int64

Для ускорения вычислений оставим тексты из 5 тем:

In [54]:
# topics = ['Экономика', 'Спорт', 'Культура', 'Путешествия', 
#           'Наука и техника']

topics = ['Экономика', 'Интернет и СМИ', 'Бизнес', 'Наука и техника']


data = data[data.topic.isin(topics)]
data.dropna(inplace=True)
len(data)

184748

### Предобработка


Все модели, с которыми мы будем работать далее, работают с предобработанными текстами, поэтому сперва проведем стандартную предобработку текста:

In [55]:
import re
import numpy as np
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from multiprocessing import Pool
# from pymystem3 import Mystem

In [56]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/pavel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
#! pip install pymorphy2

In [58]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

/home/pavel/anaconda3/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/home/pavel/anaconda3/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/home/pavel/anaconda3/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/home/pavel/anaconda3/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspe

## Дополним список стоп-слов

In [59]:
stopwords_list = stopwords.words('russian')

"это" in stopwords_list


False

In [60]:
from heapq import merge

my_stopwords_list = ['год', 'который', 'это']
stopwords_list = merge(stopwords_list, my_stopwords_list)

In [61]:
!pip install pymystem3

In [62]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]

from pymystem3 import Mystem


mystem = Mystem()
def lemmatize(words, lemmer = mystem, stopwords = stopwords_list):
    lemmas = lemmer.lemmatize(' '.join(words))
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

# def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
#     lemmas = [lemmer.parse(w)[0].normal_form for w in words]
#     return [w for w in lemmas if not w in stopwords 
#             and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

<>:1: DeprecationWarning: invalid escape sequence \w
<>:1: DeprecationWarning: invalid escape sequence \w
<ipython-input-62-a3a66f435033>:1: DeprecationWarning: invalid escape sequence \w
  words_regex = re.compile('\w+')


In [63]:
data.text.iloc[1]

'Производитель онлайн-касс «Эвотор» проанализировал данные со своих терминалов, установленных в тысячах продуктовых магазинов по стране, и рассчитал стоимость базового новогоднего стола в разных городах России. Об этом сообщает РИА Новости. Дороже всего закупка продуктов к Новому году обойдется жителям Якутии, они потратят 447 рублей на человека. Самый дешевый праздничный стол ждет жителей Оренбурга — 221 рубль на человека. Среднюю стоимость новогодних блюд по стране компания оценила в 295 рублей. В базовый набор продуктов для праздничного стола вошли оливье с колбасой, сельдь под шубой (порции по 150 граммов), два бокала шампанского, бутерброд с красной икрой, бутерброд с семгой, сырная и колбасная нарезка (по 50 граммов), три мандарина и одно яблоко. Помимо Якутии, самые высокие цены на праздничные продукты ждут жителей Приморского края, Мурманска, Магадана и Вологды. Недорогую закупку смогут сделать россияне из Ставрополя, Новосибирска, Липецка и Омска. 13 декабря стало известно, чт

In [64]:
print(preprocess(data.text.iloc[1]))

['производитель', 'онлайн', 'касса', 'эвотор', 'проанализировать', 'данный', 'свой', 'терминал', 'установленный', 'тысяча', 'продуктовый', 'магазин', 'страна', 'рассчитать', 'стоимость', 'базовый', 'новогодний', 'стол', 'разный', 'город', 'россия', 'это', 'сообщать', 'риа', 'новость', 'дорогой', 'все', 'закупка', 'продукт', 'новый', 'год', 'обходиться', 'житель', 'якутия', 'они', 'потратить', 'рубль', 'человек', 'самый', 'дешевый', 'праздничный', 'стол', 'ждать', 'житель', 'оренбург', 'рубль', 'человек', 'средний', 'стоимость', 'новогодний', 'блюдо', 'страна', 'компания', 'оценивать', 'рубль', 'базовый', 'набор', 'продукт', 'для', 'праздничный', 'стол', 'входить', 'оливье', 'колбаса', 'сельдь', 'под', 'шуба', 'порция', 'грамм', 'два', 'бокал', 'шампанский', 'бутерброд', 'красный', 'икра', 'бутерброд', 'семга', 'сырный', 'колбасный', 'нарезка', 'грамм', 'три', 'мандарин', 'один', 'яблоко', 'помимо', 'якутия', 'самый', 'высокий', 'цена', 'праздничный', 'продукт', 'ждать', 'житель', 'прим

Ниже мы оставим только случайные 10,000 текстов из нашей коллекции, чтобы ускорить вычисления:

In [65]:
data = data.sample(10000)
data.topic.value_counts()

Экономика          4266
Наука и техника    2874
Интернет и СМИ     2425
Бизнес             435 
Name: topic, dtype: int64

In [66]:
preprocessed_text = list(tqdm(map(preprocess, data['text']), total=len(data)))

In [67]:
data['text'] = preprocessed_text
data.sample(3)

,title,topic,text,tags
347967,Американские биржи закрылись заметным снижением котировок,Экономика,"[американский, торговый, площадка, итог, торги, сентябрь, закрываться, резкий, снижение, основной, биржевой, индекс, это, сообщать, agence, france, presse, опускаться, отметка, пункт, потерять, итог, день, процент, высокотехнологичный, nasdaq, упасть, процент, останавливаться, значение, пункт, индекс, dow, jones, снижаться, процент, закрытие, торги, оказываться, отметка, пункт, снижение, деловой, активность, аналитика, связывать, разногласие, повод, предоставление, финансовый, помощь, греция, сторона, государство, еврозона, сентябрь, становиться, известно, что, семь, страна, еврозона, тот, число, германия, голландия, выступать, значительный, списание, долг, греческий, облигация, сторона, частный, инвестор, время, как, другой, государство, частность, франция, выступать, против, такой, поворот, событие, опасаться, обвал, акция, банковский, сектор, негативный, настроение, биржа, отрицательно, сказываться, цена, нефть, стоимость, ноябрьский, ...]",Все
358891,"Половина россиян предпочла ""абстрактное добро"" медведевской приватизации",Экономика,"[почти, половина, россиянин, поддерживать, идея, президент, россия, дмитрий, медведев, отказ, государство, свой, доля, крупный, компания, сообщать, коммерсант, ссылка, исследование, вциом, такой, точка, зрение, высказывать, процент, опрашивать, слово, медведев, то, что, государство, должно, отказываться, только, контрольный, блокировать, пакет, акция, корпорация, получать, поддержка, лишь, процент, респондент, директор, вциом, валерий, федоров, пояснять, что, россиянин, видеть, этот, инициатива, напоминание, приватизация, госсобственность, начало, год, отношение, она, оставаться, настолько, негативный, что, вера, какой, новый, приватизация, который, мочь, быть, справедливый, нет, отмечать, федоров, исследование, вциом, показывать, что, россиянин, вызывать, отторжение, ситуация, когда, государство, являться, собственник, крупный, компания, корпорация, мнение, большинство, россиянин, утверждать, федоров, крупный, собственность, должный, быть, ...]",Все
287844,Купюры в пять тысяч рублей стали подделывать чаще остальных,Экономика,"[число, подделка, купюра, пять, тысяча, рубль, впервые, превосходить, количество, фальшивый, банкнот, номинал, тысяча, рубль, это, сообщаться, материал, банк, россия, данный, четвертый, квартал, прошлый, год, страна, выявлять, около, тысяча, фальшивый, пятитысячный, купюра, тысячерублевый, банкнота, быть, подделывать, тысяча, раз, при, это, быть, выявлять, всего, поддельный, рублевый, купюра, фальшивый, рублевый, банкнот, поддельный, рублевый, купюра, четыре, фальшивый, бумажный, червонец, кроме, то, быть, обнаруживать, поддельный, пятирублевый, монета, большой, объем, фальшивый, денежный, знак, быть, выявлять, центральный, федеральный, округ, около, тысяча, купюра, второй, место, находиться, северо, западный, федеральный, округ, тысяча, приволжский, тысяча, целое, прошлый, год, число, случай, выявление, фальшивый, деньги, страна, снижаться, год, страна, быть, выявлять, тысяча, ...]",Деньги


## Модель LDA


Первая модель, которую мы рассмотрим, LDA - латентное размещение Дирихле. Воспользуемся реализацией из библиотеки gensim.

In [68]:
# !pip install -U gensim

In [69]:
from gensim.models import *
from gensim import corpora

Модель использует векторное представление документов, например, мешок слов (bow), поэтому сперва создадим словарь:

In [70]:
dictionary = corpora.Dictionary(data['text'])

dictionary.filter_extremes(no_below = 10, no_above = 0.9) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 
dictionary.save('lenta.dict')

Векторизуем документы:

In [71]:
corpus = [dictionary.doc2bow(text) for text in data['text']]
corpora.MmCorpus.serialize('lenta.model', corpus) 

Теперь можем обучать модель:

In [ ]:
%time lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=7, chunksize=50, update_every=1, passes=2)

Посмотрим на получившиеся темы:

In [ ]:
lda.show_topics(num_topics=4, num_words=10, formatted=True)

На полученные темы можно посмотреть, изобразив их на плоскости с помощью библиотеки __pyLDAvis__. Чтобы спроецировать темы на плоскость, внутри применяется PCA-разложение.

In [ ]:
# !pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

%time vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

Посмотрим на метрики качества получившейся модели: перплексию и среднюю когерентность:

In [ ]:
print(lda.log_perplexity(corpus))


In [ ]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

In [ ]:
coherence_model_lda = CoherenceModel(model=lda, texts=data['text'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Средняя когерентность: ', coherence_lda)

Теперь можно подобрать оптимальное количество тем, опираясь на значение метрик:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
topics_list = [3, 4, 5, 6, 10, 15]
coherences = []

for num in tqdm(topics_list):
    lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=num, chunksize=50, update_every=1, passes=2)
    coherences.append(CoherenceModel(model=lda, texts=data['text'], dictionary=dictionary, coherence='c_v').get_coherence())

plt.plot(topics_list, coherences)
plt.xlabel("Число тем")
plt.ylabel("Средняя когерентность")
plt.show()

Подробнее о метриках для оценки тематических моделей:
1. [Перплексия](http://qpleple.com/perplexity-to-evaluate-topic-models/)
2. Когерентность: [один пост](http://qpleple.com/topic-coherence-to-evaluate-topic-models/), [другой пост](https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0)

In [ ]:
lda_15 = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=15, chunksize=50, update_every=1, passes=2)
vis_data = gensimvis.prepare(lda_15, corpus, dictionary)
pyLDAvis.display(vis_data)

На другие способы визуализации тематических моделей можно посмотреть [здесь](https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/#13.-t-SNE-Clustering-Chart).

## Библиотека BigARTM

Теперь попробуем построить модели с помощью библиотеки BigARTM. [Тут](https://github.com/bigartm/bigartm-book/blob/master/ARTM_example_RU.ipynb) и [тут](https://github.com/bigartm/bigartm-book/blob/master/ARTM_tutorial_RU.ipynb) можно посмотреть на примеры использования библиотеки. 

### Установка библиотеки в Google Colab

In [111]:
!apt-get --yes update
!apt-get --yes install git make cmake build-essential libboost-all-dev
!apt-get --yes install python-numpy python-pandas python-scipy
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py
!pip install protobuf tqdm wheel
!git clone --branch=stable https://github.com/bigartm/bigartm.git

ERROR! Session/line number was not unique in database. History logging moved to new session 295
shell-init: ошибка определения текущего каталога: getcwd: нет доступа к родительским каталогам: Нет такого файла или каталога
Чтение списков пакетов… Готово
E: Не удалось открыть файл блокировки /var/lib/apt/lists/lock - open (13: Отказано в доступе)
E: Невозможно заблокировать каталог /var/lib/apt/lists/
W: Проблема при удалении файла /var/cache/apt/pkgcache.bin - RemoveCaches (13: Отказано в доступе)
W: Проблема при удалении файла /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Отказано в доступе)
shell-init: ошибка определения текущего каталога: getcwd: нет доступа к родительским каталогам: Нет такого файла или каталога
E: Не удалось открыть файл блокировки /var/lib/dpkg/lock-frontend - open (13: Отказано в доступе)
E: Невозможно получить блокировку внешнего интерфейса dpkg (/var/lib/dpkg/lock-frontend); у вас есть права суперпользователя?
ERROR! Session/line number was not unique in d

In [112]:
%cd bigartm

[Errno 2] No such file or directory: 'bigartm'
/home/pavel/_GeekBrains/nlp/lesson4/bigartm/build


In [113]:
!mkdir build

shell-init: ошибка определения текущего каталога: getcwd: нет доступа к родительским каталогам: Нет такого файла или каталога
mkdir: невозможно создать каталог «build»: Нет такого файла или каталога


In [114]:
%cd build

[Errno 2] No such file or directory: 'build'
/home/pavel/_GeekBrains/nlp/lesson4/bigartm/build


In [115]:
!cmake ..

shell-init: ошибка определения текущего каталога: getcwd: нет доступа к родительским каталогам: Нет такого файла или каталога
/bin/bash: cmake: команда не найдена


In [116]:
!make

shell-init: ошибка определения текущего каталога: getcwd: нет доступа к родительским каталогам: Нет такого файла или каталога
make: getcwd: Нет такого файла или каталога
make: *** Не заданы цели и не найден make-файл.  Останов.


In [117]:
!make install

shell-init: ошибка определения текущего каталога: getcwd: нет доступа к родительским каталогам: Нет такого файла или каталога
make: getcwd: Нет такого файла или каталога
make: *** Нет правила для сборки цели «install».  Останов.


In [118]:
!pip install python/bigartm*.whl


shell-init: ошибка определения текущего каталога: getcwd: нет доступа к родительским каталогам: Нет такого файла или каталога
The folder you are executing pip from can no longer be found.


### Тематические модели в BigARTM

In [ ]:
import artm

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-119-513ce4935e65>", line 1, in <module>
    import artm
ModuleNotFoundError: No module named 'artm'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 316, in wrapped
    return f(*args, **kwargs)
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 350, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/home/pavel/anaconda3/lib/python3.8/inspect.py", line 1503, in getinnerframes
    frameinfo = (tb.tb_frame,) + getframeinfo(tb, context)
  File "/home/pavel/anaconda3/lib/python3.8/inspect.py", line 1461, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/home/pavel/anaconda3/lib/python3.8/inspect.py", line 708, in getsourcefile
    if getattr(getmodule(object, filename), '__loader__', None) is not None:
  File "/home/pavel/anaconda3/lib/python3.8/inspect.py", line 737, in getmodule
    file = getabsfile(object, _filename)
  File "/home/pavel/anaconda3/lib/python3.8/inspect.py", line 721, in getabsfile
    return os.path.n

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-119-513ce4935e65>", line 1, in <module>
    import artm
ModuleNotFoundError: No module named 'artm'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/home/pavel/anaconda3/lib/python3.8

In [ ]:
artm.version()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-119-b55518c070d6>", line 1, in <module>
    artm.version()
NameError: name 'artm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/pavel/anaconda3/lib/python3.8/sit

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-119-b55518c070d6>", line 1, in <module>
    artm.version()
NameError: name 'artm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/home/pavel/anaconda3/lib/python3.8/site-package

In [ ]:
artm.ARTM(num_topics=10).info

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 297
Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-119-a69badc5da3f>", line 1, in <module>
    artm.ARTM(num_topics=10).info
NameError: name 'artm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _f

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-119-a69badc5da3f>", line 1, in <module>
    artm.ARTM(num_topics=10).info
NameError: name 'artm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/home/pavel/anaconda3/lib/python3

Чтобы работать с данными в библиотеке ARTM нужно сперва сделать препроцессинг (мы уже сделали его ранее) и записать данные в [одном из форматов](https://bigartm.readthedocs.io/en/v0.10.1/tutorials/datasets.html), с которыми умеет работать библиотека. Мы конвертируем данные в формат vowpal wabbit:

doc1 this text will be processed not as bag of words | Some_Author

In [119]:
data['text'] = data['text'].apply(lambda x: ' '.join(x))

ERROR! Session/line number was not unique in database. History logging moved to new session 298


In [ ]:
i = 0
d = data.iloc[i]
d

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 299
Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-120-a8ba42d0dc96>", line 2, in <module>
    d = data.iloc[i]
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py", line 931, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py", line 1566, in _getitem_axis
    self._validate_integer(key, axis)
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py", line 1500, in _validate_integer
    raise IndexError("single positional indexer is out-of-bounds")
IndexError: single positional indexer is out-of-bounds

During handling of the above exception, another exception oc

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-120-a8ba42d0dc96>", line 2, in <module>
    d = data.iloc[i]
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py", line 931, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py", line 1566, in _getitem_axis
    self._validate_integer(key, axis)
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py", line 1500, in _validate_integer
    raise IndexError("single positional indexer is out-of-bounds")
IndexError: single positional indexer is out-of-bounds

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-pac

In [ ]:
f'doc{i} {d.text} |topic {d.topic}'

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-120-464e89f63fe8>", line 1, in <module>
    f'doc{i} {d.text} |topic {d.topic}'
NameError: name 'd' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/pavel/anaconda3

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2895, in _run_cell
    return runner(coro)
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/async_helpers.py", line 68, in _pseudo_sync_runner
    coro.send(None)
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3071, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3282, in run_ast_nodes
    self.showtraceback()
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2046, in showtraceback
    stb = self.InteractiveTB.structured_traceback(etype,
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
  Fi

In [120]:
def convert_to_vw_format(index, row):
    return f'doc{index} {row.text} |topic {row.topic}'

ERROR! Session/line number was not unique in database. History logging moved to new session 300


In [ ]:
with open('lenta_wv.wv', 'w') as f:
    for index, row in data.iterrows():
        vw_string = convert_to_vw_format(index, row)
        f.write(vw_string)
        f.write('\n')

ERROR! Session/line number was not unique in database. History logging moved to new session 301


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-121-39b4db551853>", line 1, in <module>
    with open('lenta_wv.wv', 'w') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'lenta_wv.wv'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_contex

Далее нужно создать объект BatchVectorizer для данных, которые мы записали в формате vowpal wabbit:

In [ ]:
batch_vectorizer = artm.BatchVectorizer(data_path='lenta_wv.wv', 
                                        data_format='vowpal_wabbit',
                                        target_folder='lenta_batches')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-121-094d5f1a9f91>", line 1, in <module>
    batch_vectorizer = artm.BatchVectorizer(data_path='lenta_wv.wv',
NameError: name 'artm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offse

Попробуем обучить модель LDA для 10 тем:

In [ ]:
%%time 
lda = artm.LDA(num_topics=10, alpha=0.1, beta=0.001, 
               cache_theta=True,
               num_document_passes=5, 
               dictionary=batch_vectorizer.dictionary)

lda.fit_offline(batch_vectorizer=batch_vectorizer, 
                num_collection_passes=10)

ERROR! Session/line number was not unique in database. History logging moved to new session 302


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/magics/execution.py", line 1315, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 1, in <module>
NameError: name 'artm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 316, in wrapped
    retu

- __fit_offline__ (офлайн алгоритм): много проходов по коллекции, один проход по документу, одно обновление матрицы $\Phi$ в конце прохода по коллекции. __Лучше использовать при работе с небольшой коллекцией__.


- __fit_online__ (онлайн алгоритм) : один проход по коллекции, несколько проходов по документу, несколько обновлений матрицы $\Phi$ в конце прохода по коллекции. __Лучше использовать при работе с большой коллекцией, или коллекцией с часто меняющимеся темами.__



Можем посмотреть на все значения перплексии (для каждой итерации по коллекции из num_collection_passes=10):

In [ ]:
lda.perplexity_value

ERROR! Session/line number was not unique in database. History logging moved to new session 303


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-121-187ce6ffa06d>", line 1, in <module>
    lda.perplexity_value
AttributeError: 'LdaModel' object has no attribute 'perplexity_value'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  Fi

In [121]:
import numpy as np

ERROR! Session/line number was not unique in database. History logging moved to new session 304


In [ ]:
np.log(lda.perplexity_last_value)

ERROR! Session/line number was not unique in database. History logging moved to new session 305


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-a47b6a546d89>", line 1, in <module>
    np.log(lda.perplexity_last_value)
AttributeError: 'LdaModel' object has no attribute 'perplexity_last_value'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_contex

Посмотрим на топ-10 токенов для каждой из тем:

In [ ]:
top_tokens = lda.get_top_tokens(num_tokens=10)
for i, token_list in enumerate(top_tokens):
     print ('Topic #{0}: {1}'.format(i, token_list))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-a6b563da43a2>", line 1, in <module>
    top_tokens = lda.get_top_tokens(num_tokens=10)
AttributeError: 'LdaModel' object has no attribute 'get_top_tokens'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_

[Более подробный ноутбук про модель LDA](https://github.com/bigartm/bigartm-book/blob/master/LDA_tutorial_RU.ipynb)

Теперь построим базовую модель pLSA:

In [ ]:
plsa = artm.ARTM(num_topics=10, 
                 dictionary=batch_vectorizer.dictionary)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 306
Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-7716d21b2190>", line 1, in <module>
    plsa = artm.ARTM(num_topics=10,
NameError: name 'artm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-7716d21b2190>", line 1, in <module>
    plsa = artm.ARTM(num_topics=10,
NameError: name 'artm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/home/pavel/anaconda3/lib/pytho

Добавим расчет перплексии:

In [ ]:
plsa.scores.add(artm.PerplexityScore(name='lenta_perplexity_score',
                 dictionary=batch_vectorizer.dictionary))

ERROR! Session/line number was not unique in database. History logging moved to new session 307


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-7c17ba3c68a4>", line 1, in <module>
    plsa.scores.add(artm.PerplexityScore(name='lenta_perplexity_score',
NameError: name 'plsa' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_of

И TopTokensScore, чтобы взглянуть на самые вероятные слова в каждой теме:

In [ ]:
plsa.scores.add(artm.TopTokensScore(name='TopTokensScore', num_tokens=6))

ERROR! Session/line number was not unique in database. History logging moved to new session 308


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-09df8116654e>", line 1, in <module>
    plsa.scores.add(artm.TopTokensScore(name='TopTokensScore', num_tokens=6))
NameError: name 'plsa' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context,

In [ ]:
plsa.fit_offline(batch_vectorizer=batch_vectorizer,
                 num_collection_passes=20)

ERROR! Session/line number was not unique in database. History logging moved to new session 309


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-7da2161b956d>", line 1, in <module>
    plsa.fit_offline(batch_vectorizer=batch_vectorizer,
NameError: name 'plsa' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/h

In [ ]:
plsa.score_tracker['lenta_perplexity_score'].value

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 310
Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-7bc4f8f08be9>", line 1, in <module>
    plsa.score_tracker['lenta_perplexity_score'].value
NameError: name 'plsa' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-7bc4f8f08be9>", line 1, in <module>
    plsa.score_tracker['lenta_perplexity_score'].value
NameError: name 'plsa' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/home/pavel/

In [ ]:
for topic_name in plsa.topic_names:
    print (topic_name + ': ', plsa.score_tracker['TopTokensScore'].last_tokens[topic_name])

ERROR! Session/line number was not unique in database. History logging moved to new session 311


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-41eb16ce808d>", line 1, in <module>
    for topic_name in plsa.topic_names:
NameError: name 'plsa' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/pavel/anacon

Обратите внимание на слова с заглавной буквы: это поле topic из оригинальных данных.

Попробуем удалить из словаря слишком редкие и слишком частые токены:

In [ ]:
batch_vectorizer.dictionary.filter(min_tf=10, max_tf=5000)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 312
Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-2de355a9510c>", line 1, in <module>
    batch_vectorizer.dictionary.filter(min_tf=10, max_tf=5000)
NameError: name 'batch_vectorizer' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py"

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-2de355a9510c>", line 1, in <module>
    batch_vectorizer.dictionary.filter(min_tf=10, max_tf=5000)
NameError: name 'batch_vectorizer' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):


In [ ]:
plsa = artm.ARTM(num_topics=10, 
                 dictionary=batch_vectorizer.dictionary)
plsa.scores.add(artm.PerplexityScore(name='lenta_perplexity_score',
                dictionary=batch_vectorizer.dictionary))

plsa.scores.add(artm.TopTokensScore(name='TopTokensScore', 
                                    num_tokens=6))
plsa.fit_offline(batch_vectorizer=batch_vectorizer, 
                 num_collection_passes=20)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 313
Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-42e813b3ecff>", line 1, in <module>
    plsa = artm.ARTM(num_topics=10,
NameError: name 'artm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-42e813b3ecff>", line 1, in <module>
    plsa = artm.ARTM(num_topics=10,
NameError: name 'artm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/home/pavel/anaconda3/lib/pytho

In [ ]:
plsa.score_tracker['lenta_perplexity_score'].value

ERROR! Session/line number was not unique in database. History logging moved to new session 314


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-7bc4f8f08be9>", line 1, in <module>
    plsa.score_tracker['lenta_perplexity_score'].value
NameError: name 'plsa' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/ho

Стало заметно лучше :)

In [ ]:
for topic_name in plsa.topic_names:
    print (topic_name + ': ', plsa.score_tracker['TopTokensScore'].last_tokens[topic_name])

ERROR! Session/line number was not unique in database. History logging moved to new session 315


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-41eb16ce808d>", line 1, in <module>
    for topic_name in plsa.topic_names:
NameError: name 'plsa' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/pavel/anacon

__Мультимодальные модели__

В библиотеке ARTM можно обучать мультимодальные модели, например, давайте попробуем обучить модель, которая будет учитывать поле topic из наших данных (мы специально записали его отдельно через | )

In [ ]:
# описываем модель и задаем веса классов
martm = artm.ARTM(num_topics=20,
                  class_ids={'@default_class': 1.0, '@topic': 5.0}, 
                  dictionary=batch_vectorizer.dictionary)

ERROR! Session/line number was not unique in database. History logging moved to new session 316


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-9d44aa8a2acb>", line 2, in <module>
    martm = artm.ARTM(num_topics=20,
NameError: name 'artm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/pavel/anaconda3

Добавим расчет метрик и регуляризаторы:

In [ ]:
martm.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score',
                                       class_id='@topic'))

martm.scores.add(artm.TopTokensScore(name='TopTokensScore',
                                     num_tokens=6))


martm.scores.add(artm.PerplexityScore(name='lenta_perplexity_score',
                dictionary=batch_vectorizer.dictionary))

martm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_def',
                    class_ids=['@default_class']))

martm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_lab',
                        class_ids=['@topic']))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 317
Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-5cf196c6367a>", line 1, in <module>
    martm.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score',
NameError: name 'martm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", li

In [ ]:
martm.fit_offline(batch_vectorizer=batch_vectorizer, 
                  num_collection_passes=10)

ERROR! Session/line number was not unique in database. History logging moved to new session 318


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-5c0e28f4658f>", line 1, in <module>
    martm.fit_offline(batch_vectorizer=batch_vectorizer,
NameError: name 'martm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "

In [ ]:
martm.score_tracker['lenta_perplexity_score'].value

ERROR! Session/line number was not unique in database. History logging moved to new session 319


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-4fe792f088e8>", line 1, in <module>
    martm.score_tracker['lenta_perplexity_score'].value
NameError: name 'martm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/

In [ ]:
for topic_name in martm.topic_names:
    print (topic_name + ': ', martm.score_tracker['TopTokensScore'].last_tokens[topic_name])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 320
Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-c930c5ba1711>", line 1, in <module>
    for topic_name in martm.topic_names:
NameError: name 'martm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    r

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-c930c5ba1711>", line 1, in <module>
    for topic_name in martm.topic_names:
NameError: name 'martm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/home/pavel/anaconda3/lib

Сравним обычную модель pLSA и модель mARTM:

In [ ]:
model_plsa = artm.ARTM(topic_names=['topic_{}'.format(i) for i in range(20)],
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                       dictionary=batch_vectorizer.dictionary)],
                       cache_theta=True)

model_martm = artm.ARTM(topic_names=['topic_{}'.format(i) for i in range(20)],
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                       dictionary=batch_vectorizer.dictionary)],
                       regularizers=[artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=-0.15)],
                       cache_theta=True,
                        class_ids={'@default_class': 1.0, '@topic': 5.0}, 
                  dictionary=batch_vectorizer.dictionary)

ERROR! Session/line number was not unique in database. History logging moved to new session 321


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3341, in run_code
    await eval(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-7b0d3772cf02>", line 1, in <module>
    model_plsa = artm.ARTM(topic_names=['topic_{}'.format(i) for i in range(20)],
NameError: name 'artm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_o

In [ ]:
model_martm.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
model_martm.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))

model_plsa.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
model_plsa.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))

ERROR! Session/line number was not unique in database. History logging moved to new session 322


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-3a78f1675406>", line 1, in <module>
    model_martm.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
NameError: name 'model_martm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_cont

In [ ]:
model_plsa.initialize(dictionary=batch_vectorizer.dictionary)
model_martm.initialize(dictionary=batch_vectorizer.dictionary)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 323
Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-7422332d3c48>", line 1, in <module>
    model_plsa.initialize(dictionary=batch_vectorizer.dictionary)
NameError: name 'model_plsa' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", l

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1192, in structured_traceback
    formatted_exception = self.format_exception_as_a_whole(etype, evalue, etb, number_of_lines_of_context,
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1150, in format_exception_as_a_whole
    last_unique, recursion_repeat = find_recursion(orig_etype, evalue, records)
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 451, in find_recursion
    return len(records), 0
TypeError: object of type 'NoneType' has no len()

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, an

In [ ]:
model_martm.num_document_passes = 1
model_plsa.num_document_passes = 1

model_martm.fit_offline(batch_vectorizer=batch_vectorizer, 
                        num_collection_passes=20)
model_plsa.fit_offline(batch_vectorizer=batch_vectorizer,
                       num_collection_passes=20)

ERROR! Session/line number was not unique in database. History logging moved to new session 324


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-322449e34bb9>", line 1, in <module>
    model_martm.num_document_passes = 1
NameError: name 'model_martm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/pavel

In [ ]:
import matplotlib.pyplot as plt

def print_measures(model_plsa, model_martm):

    print ('Perplexity: {0:.3f} (pLSA) vs. {1:.3f} (mARTM)'.format(
        model_plsa.score_tracker['PerplexityScore'].last_value,
        model_martm.score_tracker['PerplexityScore'].last_value))

    plt.plot(range(model_plsa.num_phi_updates), model_plsa.score_tracker['PerplexityScore'].value, 'b--',
             range(model_martm.num_phi_updates), model_martm.score_tracker['PerplexityScore'].value, 'r--', linewidth=2)
    
    plt.xlabel('Iterations count')
    plt.ylabel('pLSA perp. (blue), mARTM perp. (red)')
    plt.grid(True)
    plt.show()
    
print_measures(model_plsa, model_martm)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 325
Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-764ff281645c>", line 17, in <module>
    print_measures(model_plsa, model_martm)
NameError: name 'model_plsa' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1169, in get_reco

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-122-764ff281645c>", line 17, in <module>
    print_measures(model_plsa, model_martm)
NameError: name 'model_plsa' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pavel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/home/pavel/anac

ERROR! Session/line number was not unique in database. History logging moved to new session 326
